In [35]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
%run ../script/config.py

In [3]:
my_models_paths = !ls -d -1 ../data/models/*.*

my_models_paths

['../data/models/4 - Baseline - Just join triples_e089e6d2328637218601e68ee1216f624e70d78f.txt',
 '../data/models/5 - Model - Template Based - roots_6dd871aa81bea6fcaf35b71faf932af2e82e17e6.txt',
 '../data/models/5 - Model - Template Based - roots_b6a7d881636c7be572e04d2d10a4523354324356.txt']

In [4]:
models_paths = my_models_paths + SUBMISSIONS_FILEPATHS
models_paths

['../data/models/4 - Baseline - Just join triples_e089e6d2328637218601e68ee1216f624e70d78f.txt',
 '../data/models/5 - Model - Template Based - roots_6dd871aa81bea6fcaf35b71faf932af2e82e17e6.txt',
 '../data/models/5 - Model - Template Based - roots_b6a7d881636c7be572e04d2d10a4523354324356.txt',
 '../data/webnlg2017/submissions/adaptCenter/ADAPTcentreWebNLGsubmission.txt',
 '../data/webnlg2017/submissions/melbourne/final_result.txt',
 '../data/webnlg2017/submissions/pkuwriter/PKUWriter_results.txt',
 '../data/webnlg2017/submissions/tilburg/nmt_test.out.ordered',
 '../data/webnlg2017/submissions/tilburg/smt_test.out.ordered',
 '../data/webnlg2017/submissions/tilburg/template_test.out.ordered',
 '../data/webnlg2017/submissions/uit-danglt-clnlp/Submission-UIT-DANGNT-CLNLP.txt',
 '../data/webnlg2017/submissions/upf/UPF_All_sent_final.txt',
 '../data/webnlg2017/submissions/baseline_sorted.txt']

In [5]:
model_results = {}

for model_path in models_paths:
    
    with open(model_path) as f:
        
        model_results[model_path] = [line[:-1] for line in f]

In [6]:
%run ../script/webnlg.py

In [26]:
import pandas as pd
pd.set_option('max_colwidth', 1000)
from nltk.translate.bleu_score import sentence_bleu

lex = WebNLGCorpus.load("test_with_lex")

s_multiwordexpressions = lex.mdf.m_subject.str.lower().str.replace('_', ' ').str.split().tolist()
o_multiwordexpressions = lex.mdf.m_object.str.lower().str.replace('_', ' ').str.split().tolist()

all_multiwordexpressions = s_multiwordexpressions + o_multiwordexpressions
all_multiwordexpressions = list(set([tuple(s) for s in all_multiwordexpressions]))

from nltk.tokenize.mwe import MWETokenizer

t = MWETokenizer(all_multiwordexpressions)

from textacy.preprocess import preprocess_text

def preprocess(ref):
    
    ref = preprocess_text(ref, no_accents=True, lowercase=True, no_punct=True)
    
    return t.tokenize(ref.split())



def print_entry_and_models(e):
    
    print(e)
    print('\n')
    
    idx = int(e.idx().split('_')[1])
    
    references = [preprocess(ref) for ref in e.ldf.ltext.tolist()]
    
    texts = []
    bleus = []
    
    for model_path in models_paths:
        
        model_text = model_results[model_path][idx]
        
        bleu = sentence_bleu(references, preprocess(model_text))
        
        texts.append(model_text)
        bleus.append(bleu)
        
    return pd.DataFrame({'models': models_paths,
                         'texts': texts,
                         'bleu': bleus}).sort_values('bleu', ascending=False)

In [8]:
test_with_lex = WebNLGCorpus.load('test_with_lex')

In [84]:
e = test_with_lex.sample()

df = print_entry_and_models(e)

Triple info: {'category': 'Airport', 'eid': 'Id286', 'idx': '0_285', 'ntriples': 2}

	Modified triples:

Adolfo_Suárez_Madrid–Barajas_Airport | elevationAboveTheSeaLevel_(in_metres) | 610.0
Adolfo_Suárez_Madrid–Barajas_Airport | location | "Madrid, Paracuellos de Jarama, San Sebastián de los Reyes and Alcobendas"


	Lexicalizations:

Adolfo Suarez Madrid-Barajas Airport at Madrid, Paracuellos de Jarama, San Sebastián de los Reyes and Alcobendas is 610 metres above sea level..
Adolfo Suarez Madrid-Barajas Airport is located in Madrid, Paracuellos de Jarama, San Sebastian de los Reyes and Alcobendas and lies 610 metres above sea level.
Adolfo Suarez Madrid-Barajas airport is located at Madrid, Paracuellos de Jarama, San Sebastián de los Reyes and Alcobendas and is elevated 610 metres above sea level.




In [85]:
df

,bleu,models,texts
11,0.865732,../data/webnlg2017/submissions/baseline_sorted.txt,"adolfo suarez madrid - barajas airport is located in madrid , paracuellos de jarama , san sebastian de los reyes and alcobendas and is 610 . 0 metres above sea level ."
6,0.829187,../data/webnlg2017/submissions/tilburg/nmt_test.out.ordered,"adolfo suárez madrid -- barajas airport in madrid , paracuellos de jarama , san sebastián de los reyes and alcobendas is 610 metres metres above sea level ."
8,0.829187,../data/webnlg2017/submissions/tilburg/template_test.out.ordered,"adolfo suárez madrid -- barajas airport in madrid , paracuellos de jarama , san sebastián de los reyes and alcobendas is 610 metres metres above sea level ."
4,0.789014,../data/webnlg2017/submissions/melbourne/final_result.txt,"adolfo suárez madrid–barajas airport in madrid, paracuellos de jarama, san sebastián de los reyes and alcobendas is 610.0 metres above sea level ."
5,0.756156,../data/webnlg2017/submissions/pkuwriter/PKUWriter_results.txt,"adolfo suárez madrid – barajas airport is located in madrid , paracuellos de jarama , san sebastián de los reyes and alcobendas , the airport is 610 . 0 metres above sea level ."
7,0.689875,../data/webnlg2017/submissions/tilburg/smt_test.out.ordered,"airport is 610 metres above sea level is in madrid , paracuellos de jarama , san sebastián de los reyes and alcobendas"
2,0.674432,../data/models/5 - Model - Template Based - roots_b6a7d881636c7be572e04d2d10a4523354324356.txt,"Adolfo Suárez Madrid–Barajas Airport is elevated 610.0 metres above sea level. Adolfo Suárez Madrid–Barajas Airport is located in Madrid, Paracuellos de Jarama, San Sebastián de los Reyes and Alcobendas."
1,0.673521,../data/models/5 - Model - Template Based - roots_6dd871aa81bea6fcaf35b71faf932af2e82e17e6.txt,"Adolfo Suárez Madrid–Barajas Airport is 610.0 metres above sea level. Adolfo Suárez Madrid–Barajas Airport is located in Madrid, Paracuellos de Jarama, San Sebastián de los Reyes and Alcobendas."
9,0.604931,../data/webnlg2017/submissions/uit-danglt-clnlp/Submission-UIT-DANGNT-CLNLP.txt,"adolfo suárez madrid – barajas airport , located in madrid , paracuellos de jarama , san sebastián de los reyes and alcobendas , is at an elevation of 610.0 from sea level ."
3,0.593798,../data/webnlg2017/submissions/adaptCenter/ADAPTcentreWebNLGsubmission.txt,"adolfo suarez madrid-barajas airport is located at madrid , paracuellos de jarama , san sebastián de los reyes and alcobendas and is 610 metres above sea level . the airport is located at madrid , paracuellos de jarama , san sebastián de los reyes and alcobendas ."


In [29]:
%run ../script/data_alignment.py

import spacy

nlp = spacy.load('en_core_web_lg')

In [81]:
%run ../script/data_alignment.py

In [82]:
da = RootDataAlignmentModel(similarity_metric=similarity.token_sort_ratio)

In [83]:
first_lexicalization = e.ldf.ltext.values.tolist()[1]
# uses the first triple
first_triple = e.mdf[['m_subject', 'm_object']].to_dict(orient='records')[0]

da.render_aligned(nlp(first_lexicalization), first_triple)